In [3]:
import scipy.io
import scipy.signal as signal
from scipy.signal import butter, lfilter
import numpy as np
import pandas as pd
import mne
from mne.preprocessing import ICA
from mne.channels import make_standard_montage
%matplotlib qt
%gui qt
from time import time
from matplotlib import pyplot as plt
from mne.preprocessing import find_eog_events, regress_artifact
from mne.datasets import sample
from mne.viz import plot_raw
from mne.preprocessing import corrmap
from mne.viz import plot_raw
from meegkit.asr import ASR
from meegkit.utils.matrix import sliding_window
from scipy.signal import butter, lfilter

In [4]:

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def apply_asr(raw, sfreq=250, cutoff=2.5, blocksize=100, win_len=0.5,
        win_overlap=0.66, max_dropout_fraction=0.1,
        min_clean_fraction=0.25, name='asrfilter', method='euclid',
        estimator='scm'):
   
    h, w = raw.shape
    
    # t = int(raw.shape[1] / sfreq)
    asr = ASR(sfreq=sfreq, cutoff=cutoff, blocksize=blocksize, win_len=win_len,
        win_overlap=win_overlap, max_dropout_fraction=max_dropout_fraction,
        min_clean_fraction=min_clean_fraction, name=name, method=method,
        estimator=estimator)
        
        # method='euclid'
    # train_idx = np.arange(0 * sfreq, t * sfreq, dtype=int)
    _, sample_mask = asr.fit(raw)
    
    # Apply filter using sliding (non-overlapping) windows
    X = sliding_window(raw, window=int(sfreq), step=int(sfreq))
    Y = np.zeros_like(X)
    for i in range(X.shape[1]):
        Y[:, i, :] = asr.transform(X[:, i, :])
    
    raw = X.reshape(h, -1)  # reshape to (n_chans, n_times)
    clean = Y.reshape(h, -1)
    return raw, clean

import pyedflib



In [ ]:
data_raw = scipy.io.loadmat('E:\\software\\EEG+Neuroguide report\\31_khanzade\\khanzade.mat')
f=data_raw['EEG_Data']
g=np.array((f))
filter_signal= butter_bandpass_filter(g, 4, 50, 250)
signal3, filter_asr_signal3 = apply_asr(filter_signal, sfreq=250)
from tempfile import TemporaryFile
pre_khanzade = TemporaryFile()
np.save(pre_khanzade, filter_asr_signal3)

In [5]:
data_raw = scipy.io.loadmat('E:\\software\\EEG+Neuroguide report\\31_khanzade\\khanzade.mat')
f=data_raw['EEG_Data']
g=np.array((f))
filter_signal= butter_bandpass_filter(g, 4, 50, 250)
signal3, filter_asr_signal3 = apply_asr(filter_signal, sfreq=250)
from tempfile import TemporaryFile
pre_khanzade = TemporaryFile()
np.save(pre_khanzade, filter_asr_signal3)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\software\\EEG+Neuroguide report\\31_khanzade\\khanzade.mat'